In [1]:
import findspark
findspark.init('/home/mj1e16/miniconda2/lib/python2.7/site-packages/pyspark')

In [2]:
import astroprov
import numpy as np
import os

from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.table import Table
from astropy.table import Column

import collections
import subprocess

import time
from multiprocessing import Pool
from multiprocessing.pool import ThreadPool

from functools import partial

from astropy.io import ascii
#from sqlalchemy import create_engine
import sqlite3
import mysql.connector
from pandas import DataFrame

import mysql.connector
from mysql.connector import Error
from mysql.connector import pooling

from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark import SparkConf
import datetime
from sqlalchemy import create_engine


In [3]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/mj1e16/miniconda2/lib/python2.7/site-packages/pyspark/mysql-connector-java_8.0.16-1ubuntu16.04_all/usr/share/java/mysql-connector-java-8.0.16.jar  pyspark-shell'
config = SparkConf().setAll([('spark.executor.cores', '6'),('spark.cores.max', '6'),('spark.driver.memory','1g'),('spark.executor.memory', '500m')])


In [4]:
sc = SparkContext(appName='App',conf=config)
sqlContext = SQLContext(sc)
dataframe_mysql = sqlContext.read.format("jdbc").option("url", "jdbc:mysql://localhost/Kepler").option("driver", "com.mysql.jdbc.Driver").option("dbtable", "starlist").option("user", "mj1e16").option("password", "[sqlT1G3R]").load()
dataframe_mysql.registerTempTable('starlist')

In [5]:
def innerJoin(tableName,provDir,diffSize=1):
    temptableName = tableName[0:5]+tableName[-1]
    df = sqlContext.sql("SELECT NUMBER, EXT_NUMBER, FLUX_ISO, FLUXERR_ISO, BACKGROUND, THRESHOLD, FLUX_MAX, XPEAK_IMAGE, YPEAK_IMAGE,  X_IMAGE, Y_IMAGE, FWHM_IMAGE, ELLIPTICITY FROM {} INNER JOIN starlist ON {}.X_IMAGE BETWEEN starlist.X_POS_MIN AND starlist.X_POS_MAX AND {}.Y_IMAGE BETWEEN starlist.Y_POS_MIN AND starlist.Y_POS_MAX".format(temptableName,temptableName,temptableName))
    df.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='result_{}'.format(tableName),user='mj1e16',password='[sqlT1G3R]').mode('append').save()
    sqlContext.dropTempTable(temptableName)
    astroprov.provcall([tableName,'starlist'],['result_{}'.format(tableName)],"innerJoin_Python2Python_SQ_tmpl.provn","innerJoin",provDir)


def makeConfig(valList,provDir,tableName='table',defaultDir='/home/mj1e16/sextractor/sextractor-master/config/',attributeList=['DETECT_THRESH','DETECT_MINAREA','BACK_FILTERSIZE','BACK_SIZE']):
    
    #workAroundList = [16,32,64,128,256,512]
    workAroundList = [1,3,5,7,9,11]
    ident = workAroundList.index(valList[-1])
    tableName = tableName[ident] # could just return a letter?
    with open(defaultDir+'/default.sex','r') as f:
        data  = f.read()
    
    catLocFinder = 'CATALOG_NAME'
    catname = 'test'+str(ident)+'.cat'
    nameLoc = data.find(catLocFinder) + len(catLocFinder)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' '+catname+' ' + data[endLoc:]
    data = newData 
        
    for x in range(len(valList)):
        nameLoc = data.find(attributeList[x]) + len(attributeList[x])
        endLoc = data[nameLoc:].find('#') + nameLoc
        newData = data[:nameLoc] + ' '+str(valList[x])+' ' + data[endLoc:]
        data = newData    
    
    
    confName = 'default_'+str(ident)+'.sex'
    with open(defaultDir+confName,'w') as f:
        f.write(data)
    astroprov.provcall(valList,[confName],"makeConfig_Python2Python_SQ_tmpl.provn","makeConfig",provDir)
    return(confName,tableName,catname)



def findObjects(confName,tableName,catName,provDir,defaultDir='/home/mj1e16/sextractor/sextractor-master/config/',imagename='/home/mj1e16/iraf/editedImage5000.fits'):
    
    astroprov.provcall([confName,'/home/mj1e16/iraf/editedImage5000.fits'],[tableName],"findObjects_Python2DaoStarfidner_SQ_tmpl.provn","findObjects",provDir)

    os.chdir(defaultDir)
    subprocess.call(['sex',imagename,'-c',confName])
    assoc = Table.read(catName,format='ascii.sextractor')
    df = assoc.to_pandas()
    df_spark = sqlContext.createDataFrame(df)
    temptableName = tableName[0:5]+tableName[-1]
    #print(temptableName)
    df_spark.registerTempTable(temptableName)

    #return df

def alltogethernow(valList,tableName,IMAGE,provDir):
    Names = makeConfig(valList,provDir,tableName=tableName)
    findObjects(Names[0],Names[1],Names[2],provDir,imagename=IMAGE) # confName tabName catname
    innerJoin(Names[1],provDir)


In [6]:
valList = [np.linspace(1,10,10),np.linspace(1,9,9),np.linspace(8,128,16),[1,3,5,7,9,11]] # astro start

In [7]:
def evaluateImage(valList,simImage,provDir,smallName='Poshak',atList=['DETECT_THRESH','DETECT_MINAREA','BACK_SIZE','BACK_FILTERSIZE']):

    p = ThreadPool(6)
    t0 = time.time()
    for x0 in range(len(valList[0])):
        for x1 in range(len(valList[1])):
                for x2 in range(len(valList[2])):
                    name = smallName+'_'+str(x0)+'_'+str(x1)+'_'+str(x2)+'_'
                    fullname = [name]*len(valList[3])
                    fullValList = []
                    for x3 in range(len(valList[3])):
                        fullname[x3] += str(x3)
                        print(fullname[x3])
                        fullValList.append([valList[0][x0],valList[1][x1],valList[2][x2],valList[3][x3]])

                        #alltogethernow(fullValList[x4],fullname,imageName,provDir)
                    try:
                        p.map(partial(alltogethernow,tableName=fullname,IMAGE=simImage,provDir=provDir),fullValList)
                    except RuntimeError:
                        with open('redos','a') as f:
                            bigString = simImage+','+str(fullname)+','+str(fullValList)
                            f.write(bigString)
                        print('Redo with valList = ',fullValList)
                        pass

    #def alltogethernow(valList,tableName,IMAGE,provDir):

    print(time.time()-t0)

In [ ]:
### Make function to replace starlist
evaluateImage(valList,'/home/mj1e16/Simages/diff_44_1_alt.fits','/home/mj1e16/keplerPhotometry/provDump/44/selectiveMedian/',smallName='Poshak_44_selMed')

Poshak_44_selMed_0_0_0_0
Poshak_44_selMed_0_0_0_1
Poshak_44_selMed_0_0_0_2
Poshak_44_selMed_0_0_0_3
Poshak_44_selMed_0_0_0_4
Poshak_44_selMed_0_0_0_5
Poshak_44_selMed_0_0_1_0
Poshak_44_selMed_0_0_1_1
Poshak_44_selMed_0_0_1_2
Poshak_44_selMed_0_0_1_3
Poshak_44_selMed_0_0_1_4
Poshak_44_selMed_0_0_1_5
Poshak_44_selMed_0_0_2_0
Poshak_44_selMed_0_0_2_1
Poshak_44_selMed_0_0_2_2
Poshak_44_selMed_0_0_2_3
Poshak_44_selMed_0_0_2_4
Poshak_44_selMed_0_0_2_5
Poshak_44_selMed_0_0_3_0
Poshak_44_selMed_0_0_3_1
Poshak_44_selMed_0_0_3_2
Poshak_44_selMed_0_0_3_3
Poshak_44_selMed_0_0_3_4
Poshak_44_selMed_0_0_3_5
Poshak_44_selMed_0_0_4_0
Poshak_44_selMed_0_0_4_1
Poshak_44_selMed_0_0_4_2
Poshak_44_selMed_0_0_4_3
Poshak_44_selMed_0_0_4_4
Poshak_44_selMed_0_0_4_5
Poshak_44_selMed_0_0_5_0
Poshak_44_selMed_0_0_5_1
Poshak_44_selMed_0_0_5_2
Poshak_44_selMed_0_0_5_3
Poshak_44_selMed_0_0_5_4
Poshak_44_selMed_0_0_5_5
Poshak_44_selMed_0_0_6_0
Poshak_44_selMed_0_0_6_1
Poshak_44_selMed_0_0_6_2
Poshak_44_selMed_0_0_6_3
